In [1]:
import random
import os
import math
import time
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

# Specify an input embedding and a directory in which the folder containing clusters will be generated
infile = "./case_embedding.csv"
outdir = "./clusters_case" + str(time.strftime("%Y-%m-%d_%H-%M-%S")) + "/"

# Local/Global particle velocity weights & movememnt weight
cl = 0.095    # ~0.095
cg = 0.125    # ~0.125
W = 0.4    # ~0.4

r_seed = 13
tmpinf = 1000.0    # Initial fitness for each particle (lower fitness score == better fit)
fit_threshold = 0.000001    # ~0.000001

# Random Float Range (for velocity weights)
start_w = -1.0
stop_w = 1.0

num_iters = 1000  # Number of Particle Position Changes (Maximum) ~100
num_parts = 100       # Number of Particles/Centroids (10, 20, 30, 50)

In [2]:
def cosine_similarity(a,b):
    ans=cosine(a,b)
    return ans

In [3]:
def full_euc_dis(a, b):    # Euclidian distance of full vector
    l = len(a)
    arg = 0
    
    for i in range(l):
        arg += (a[i]-b[i])**2
        
    return(math.sqrt(arg))

In [4]:
def euc_dis(a, b):    # Euclidian distance of individual points (used for pointwise PSO fitness)
    arg = float((a-b)**2)
        
    return(math.sqrt(arg))

In [5]:
class particle:
    def __init__(self, pos, c_shape):
        self.cur_pos = pd.Series(data=list(pos))
        self.cur_fit = list([tmpinf]*c_shape)
        
        self.vel = pd.Series(data=list([0.0]*c_shape))  # Initial velocity of 0
        
        self.best_pos = pd.Series(data=list(pos))
        self.best_fit = list([tmpinf]*c_shape)
        
        self.members = list()    # dict of dataframe indices for the surrounding cluster members & the dist to centroid
    
    # Pointwise fitness function for training centroids
    def fit_func(self, embdf):
        r_shape, c_shape = embdf.shape
        fit = list([0.0]*c_shape)
        
        for i in range(r_shape):    # For each submission,
            curr_pos = self.cur_pos.to_numpy()
            embd_pos = embdf.loc[i].to_numpy()
            
            for j in range(c_shape):    # For each embedded dimension,
                #fit[j] += cosine_similarity(curr_pos[j], embd_pos[j])
                fit[j] += euc_dis(curr_pos[j], embd_pos[j])
        
        for k in range(c_shape):    # Average each dimension of the fitness for the number of submissions
            fit[k] /= r_shape
        
        return list(fit)
        
    def change_pos(self):
        temp_pos = self.cur_pos.to_numpy() + self.vel.to_numpy()
        self.cur_pos = pd.Series(data=list(temp_pos))
    
    def change_vel(self, g_best_pos):        
        l_new_vel = (cl)*(self.best_pos.to_numpy() - self.cur_pos.to_numpy())
        g_new_vel = (cg)*(g_best_pos.to_numpy() - self.cur_pos.to_numpy())
        
        temp_vel = (W*self.vel.to_numpy()) + l_new_vel + g_new_vel
        self.vel = pd.Series(data=list(temp_vel))
        
        self.change_pos()

In [6]:
class swarm:
    def __init__(self, embdf, num_particles):
        random.seed(r_seed)
        
        r_shape, c_shape = embdf.shape
        tempParts = list()
        
        for i in range(num_particles):
            rand_row = random.randint(0, r_shape-1)
            tempParts.append(particle(pos=list(embdf.loc[rand_row].to_numpy()), c_shape=c_shape))
        
        self.particles = list(tempParts)
        self.g_best_pos = pd.Series(data=list([0.0]*c_shape))
        self.g_best_fit = list([tmpinf]*c_shape)
        self.num_dims = int(c_shape)
        
        #  A list of values ranging from (0 - num_particles)
        self.clus_assign = [-1]*r_shape    # Each entry is a submission, each value is the cluster assignment
        self.clus_count = [0]*num_particles
        self.total_num_subs = 0
        
        self.topic_nums = dict()
        
        self.actual_cents = [-1]*num_particles
    
    def train(self, embdf, num_iters):
        r_shape, c_shape = embdf.shape
        done = False
        
        for it in range(num_iters):
            if done == True:
                break
            
            flag = True
            
            for i, part in enumerate(self.particles):
                temp_fit = part.fit_func(embdf)
                
                print('iter', it, 'i', i, 'fit(s)', float(sum(part.cur_fit)/r_shape), float(sum(temp_fit)/r_shape))    #TEMP
                
                for j in range(c_shape):    # For each embedded dimension,
                    if part.cur_fit[j] > temp_fit[j]:
                        if abs(part.cur_fit[j] - temp_fit[j]) < fit_threshold:
                            flag = True

                        else:
                            part.cur_fit[j] = float(temp_fit[j])
                            part.best_pos[j] = float(part.cur_pos[j])
                            flag = False

                    if self.g_best_fit[j] > temp_fit[j]:
                        if abs(self.g_best_fit[j] - temp_fit[j]) < fit_threshold:
                            flag = True

                        else:
                            self.g_best_fit[j] = float(temp_fit[j])
                            self.g_best_pos[j] = float(part.cur_pos[j])
                            flag = False
            
            if flag:
                done = True
            
            for i, part in enumerate(self.particles):
                part.change_vel(self.g_best_pos)
        
        print('iter', it, "Gbest fitness", self.g_best_fit)
    
    # Full vector evaluation for document cluster member assignment
    def test(self, iddf, embdf):  
        r_shape, c_shape = embdf.shape

        for i in range(r_shape):    # For each document,
            dist = tmpinf
            
            for k, part in enumerate(self.particles):
                temp_dist = cosine_similarity(part.cur_pos.to_numpy(), embdf.loc[i].to_numpy())
                #temp_dist = full_euc_dis(part.cur_pos.to_numpy(), embdf.loc[i].to_numpy())

                # check the best suitable centroid
                if (dist > temp_dist):
                    self.clus_assign[i] = k
                    dist = temp_dist
            
            self.particles[self.clus_assign[i]].members.append(iddf.loc[i])
        
        # count the number of submissions per cluster
        for i in range(r_shape):
            self.clus_count[self.clus_assign[i]] += 1
            self.total_num_subs += 1

        print('count', self.clus_count)

In [7]:
def generate_clusters(indf, outdir):
    tp_vecs = list()    # List of topic vectors for each particle to output
    
    # Split input into embedding and metadata dataframes
    embdf = indf[indf.columns[2:]]
    embdf = embdf.astype(float)
    metadf = indf[indf.columns[:2]]
    
    # Separate ID column for cluster generation and document mapping
    iddf = metadf['id']
    
    PSO = swarm(embdf, num_parts)
    PSO.train(embdf, num_iters)
    PSO.test(iddf, embdf)
    
    try:
        os.mkdir(outdir)

    except (FileExistsError):
        pass
    
    for k, part in enumerate(PSO.particles):
        outfile = outdir + "clus_" + str(k) + ": [" + str(len(part.members)) + "_docs].csv"
        print("particle members",part.members)
        clusdf = indf[indf['id'].isin(part.members)]
        clusdf.to_csv(outfile, index=False, encoding="utf-8")
        
        tp_vecs.append(part.cur_pos.to_numpy())    # Record the topic vector for each particle
    
    # Write topic vectors relative to cluster numbers as a CSV for document completion prediction analysis
    cols = list(range(int(tp_vecs[0].shape[0])))
    tp_df = pd.DataFrame(data=tp_vecs, columns=cols)
    
    tp_df.to_csv(str(outdir + "topic_vectors.csv"), index=True, index_label="clus_num", encoding="utf-8")

In [8]:
indf = pd.read_csv(infile, encoding='utf-8')
indf = indf[pd.to_numeric(indf['0'], errors='coerce').notnull()]
indf = indf.reset_index()
indf = indf.drop(columns=['index'])
indf.insert(loc=0, column='id', value=np.arange(len(indf)))
indf

,id,text,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
0,0,Patient fell at home.,0.525626,0.677345,0.540085,0.443428,0.587985,0.574895,0.533140,0.721757,...,0.392471,0.580851,0.558423,0.585362,0.458769,0.711421,0.180124,0.529434,0.436539,0.627444
1,1,/ flex/ex at ankle.,0.241540,0.446862,0.469783,0.429766,0.666816,0.514777,0.684461,0.610812,...,0.493009,0.529302,0.366739,0.549317,0.393767,0.643230,0.563106,0.417858,0.392481,0.429932
2,2,See attached icon.,0.446700,0.442859,0.300642,0.483094,0.664171,0.565392,0.484976,0.425587,...,0.366638,0.415025,0.571503,0.465555,0.631344,0.589719,0.501487,0.598771,0.506597,0.455115
3,3,The nasal bones are intact.,0.533445,0.522022,0.577971,0.273175,0.491703,0.541962,0.438486,0.428465,...,0.120753,0.469756,0.445602,0.588247,0.516588,0.646629,0.364220,0.444125,0.532299,0.442413
4,4,Still unable to contact.,0.355810,0.448629,0.499945,0.326376,0.721293,0.429723,0.357022,0.377097,...,0.454149,0.396839,0.561842,0.389477,0.568566,0.362305,0.522260,0.382967,0.393214,0.536416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570150,570150,Vascular calcification.,0.636517,0.579212,0.447155,0.435646,0.501272,0.351761,0.564179,0.565306,...,0.134406,0.429600,0.615561,0.658789,0.581636,0.590343,0.332434,0.623284,0.651573,0.258769
570151,570151,USUALLY USES A CANE AT HOME BUT HAD TO GET OU...,0.422327,0.510618,0.394199,0.556117,0.586783,0.519537,0.479301,0.499568,...,0.462218,0.450073,0.546485,0.442449,0.521720,0.515545,0.406746,0.441219,0.415413,0.514141
570152,570152,No acute skull fracture is seen.,0.562470,0.648922,0.489147,0.411536,0.610383,0.583875,0.290394,0.457097,...,0.412161,0.578461,0.623547,0.636212,0.628531,0.557545,0.627855,0.361554,0.514662,0.475945
570153,570153,"VOICE ID / DOC ID LAL,MI/sa D: // :: ...",0.423318,0.477936,0.519891,0.352037,0.606809,0.592510,0.530173,0.461302,...,0.382832,0.528713,0.487460,0.522070,0.643532,0.488422,0.470535,0.486947,0.387388,0.457677


In [12]:
indf_10000=indf.head(10000)
indf_10000

,id,text,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
0,0,Patient fell at home.,0.525626,0.677345,0.540085,0.443428,0.587985,0.574895,0.533140,0.721757,...,0.392471,0.580851,0.558423,0.585362,0.458769,0.711421,0.180124,0.529434,0.436539,0.627444
1,1,/ flex/ex at ankle.,0.241540,0.446862,0.469783,0.429766,0.666816,0.514777,0.684461,0.610812,...,0.493009,0.529302,0.366739,0.549317,0.393767,0.643230,0.563106,0.417858,0.392481,0.429932
2,2,See attached icon.,0.446700,0.442859,0.300642,0.483094,0.664171,0.565392,0.484976,0.425587,...,0.366638,0.415025,0.571503,0.465555,0.631344,0.589719,0.501487,0.598771,0.506597,0.455115
3,3,The nasal bones are intact.,0.533445,0.522022,0.577971,0.273175,0.491703,0.541962,0.438486,0.428465,...,0.120753,0.469756,0.445602,0.588247,0.516588,0.646629,0.364220,0.444125,0.532299,0.442413
4,4,Still unable to contact.,0.355810,0.448629,0.499945,0.326376,0.721293,0.429723,0.357022,0.377097,...,0.454149,0.396839,0.561842,0.389477,0.568566,0.362305,0.522260,0.382967,0.393214,0.536416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,Would increase the dose of lisinopril and/or h...,0.489395,0.446677,0.439956,0.439343,0.555340,0.483886,0.484283,0.587757,...,0.410833,0.634996,0.421598,0.407079,0.452085,0.525622,0.390769,0.373569,0.382034,0.464727
9996,9996,"- Cont ASA, BB, statin.",0.244531,0.530350,0.358849,0.362884,0.652721,0.649448,0.627380,0.649460,...,0.592135,0.572250,0.430362,0.550024,0.580999,0.596486,0.456957,0.417021,0.447141,0.469066
9997,9997,He stated that his family would be in later to...,0.509332,0.594433,0.510910,0.404397,0.646998,0.574908,0.533295,0.447422,...,0.516334,0.492860,0.521099,0.431388,0.568398,0.458487,0.324247,0.573444,0.390057,0.451974
9998,9998,she ambulates using his cane.,0.429214,0.596503,0.368407,0.529643,0.425013,0.556603,0.579129,0.581242,...,0.476820,0.364077,0.653987,0.428890,0.467137,0.602244,0.316858,0.468813,0.600573,0.374538


In [13]:
# Read the Embedded CSV for the Submissions (Input File)
#indf = pd.read_csv(infile, encoding='utf-8')
outdir = "./clusters_case" + str(time.strftime("%Y-%m-%d_%H-%M-%S")) + "/"
generate_clusters(indf_10000, outdir)

iter 0 i 0 fit(s) 38.4 0.0036879623520042477
iter 0 i 1 fit(s) 38.4 0.004732323535645387
iter 0 i 2 fit(s) 38.4 0.003922458702032965
iter 0 i 3 fit(s) 38.4 0.004226726824358642
iter 0 i 4 fit(s) 38.4 0.004022377194094942
iter 0 i 5 fit(s) 38.4 0.0032381578509803645
iter 0 i 6 fit(s) 38.4 0.0046777942552063915
iter 0 i 7 fit(s) 38.4 0.002973921837041665
iter 0 i 8 fit(s) 38.4 0.003742980598612758
iter 0 i 9 fit(s) 38.4 0.0041520509092696604
iter 0 i 10 fit(s) 38.4 0.003206086676579382
iter 0 i 11 fit(s) 38.4 0.0038181573042556406
iter 0 i 12 fit(s) 38.4 0.0031003874687208105
iter 0 i 13 fit(s) 38.4 0.00330044280144254
iter 0 i 14 fit(s) 38.4 0.003933421790603309
iter 0 i 15 fit(s) 38.4 0.0035422966035092373
iter 0 i 16 fit(s) 38.4 0.0038248167930159055
iter 0 i 17 fit(s) 38.4 0.0036641231743096796
iter 0 i 18 fit(s) 38.4 0.0037036110833917803
iter 0 i 19 fit(s) 38.4 0.004266663793109101
iter 0 i 20 fit(s) 38.4 0.004062626972110907
iter 0 i 21 fit(s) 38.4 0.004087662323985685
iter 0 i 22

iter 1 i 60 fit(s) 0.0037274327751325186 0.003514782365104702
iter 1 i 61 fit(s) 0.0035819484376773338 0.0033948570181530457
iter 1 i 62 fit(s) 0.004366677072853199 0.004039263566062227
iter 1 i 63 fit(s) 0.0039239130279310995 0.0036756283556194
iter 1 i 64 fit(s) 0.0035038675035672243 0.0033311032483759545
iter 1 i 65 fit(s) 0.00393257612311812 0.0036808211078232754
iter 1 i 66 fit(s) 0.00406478566963379 0.0037895159955578734
iter 1 i 67 fit(s) 0.004421795191194037 0.004088168956287704
iter 1 i 68 fit(s) 0.004327380960774963 0.00400720502808875
iter 1 i 69 fit(s) 0.0045070830058048135 0.004154516656175958
iter 1 i 70 fit(s) 0.003883192297198347 0.003639969764887637
iter 1 i 71 fit(s) 0.004961453181758124 0.004535395439423375
iter 1 i 72 fit(s) 0.003233284472886933 0.003113575001969391
iter 1 i 73 fit(s) 0.004468713621379271 0.004126894975641758
iter 1 i 74 fit(s) 0.0033439738378403185 0.0032029988396558226
iter 1 i 75 fit(s) 0.004310173113583722 0.003990328477237381
iter 1 i 76 fit(s)

iter 2 i 93 fit(s) 0.0037966101136119503 0.003473617230450931
iter 2 i 94 fit(s) 0.004227189732604567 0.003792015704440001
iter 2 i 95 fit(s) 0.003348454431717031 0.0031457383802513954
iter 2 i 96 fit(s) 0.0038345056389879105 0.003500723470375442
iter 2 i 97 fit(s) 0.0039479479073533325 0.0035854098393672744
iter 2 i 98 fit(s) 0.0033681082328335323 0.003159802942483606
iter 2 i 99 fit(s) 0.0038370085693697617 0.0035053299476104686
iter 3 i 0 fit(s) 0.0032418110298378816 0.0030414342168346023
iter 3 i 1 fit(s) 0.003892932643470841 0.003491618524487837
iter 3 i 2 fit(s) 0.003382258988287912 0.003135858743967741
iter 3 i 3 fit(s) 0.003570100079877919 0.003264120651308012
iter 3 i 4 fit(s) 0.003437612378811856 0.0031711684600216478
iter 3 i 5 fit(s) 0.00298192851852911 0.002870473400068853
iter 3 i 6 fit(s) 0.0038407653694324615 0.0034468864104231843
iter 3 i 7 fit(s) 0.0028332563763301933 0.0027741001027673947
iter 3 i 8 fit(s) 0.0032700255415712425 0.003058603578026926
iter 3 i 9 fit(s) 

iter 4 i 26 fit(s) 0.003021687901201886 0.0028850498318218802
iter 4 i 27 fit(s) 0.0028181295955576695 0.002760296412542765
iter 4 i 28 fit(s) 0.0029725402142586456 0.0028555747633105706
iter 4 i 29 fit(s) 0.003225249204471272 0.0030147654398973807
iter 4 i 30 fit(s) 0.0028108791894082483 0.002755874611791321
iter 4 i 31 fit(s) 0.0030232097817871175 0.00288604680317752
iter 4 i 32 fit(s) 0.002734091748940429 0.002710047820547015
iter 4 i 33 fit(s) 0.0027899953559895677 0.0027435543875940667
iter 4 i 34 fit(s) 0.0029358880279968347 0.0028325423327469116
iter 4 i 35 fit(s) 0.002748341814730208 0.0027185013403657667
iter 4 i 36 fit(s) 0.002791059399142267 0.0027442545442546555
iter 4 i 37 fit(s) 0.003404371001882703 0.003128525842275409
iter 4 i 38 fit(s) 0.0034936687951487824 0.003186474830939134
iter 4 i 39 fit(s) 0.0030587280894833605 0.0029086596831616562
iter 4 i 40 fit(s) 0.003365494394065679 0.0031021868870584435
iter 4 i 41 fit(s) 0.003211192125426242 0.0030043632940409547
iter 4 

iter 5 i 59 fit(s) 0.002802958785305102 0.0027486932539889527
iter 5 i 60 fit(s) 0.002908747854158358 0.0028112329829444722
iter 5 i 61 fit(s) 0.002871013141928898 0.002788653237358741
iter 5 i 62 fit(s) 0.003074224974974943 0.0029110542489845085
iter 5 i 63 fit(s) 0.002958623840644106 0.0028407709885825364
iter 5 i 64 fit(s) 0.002852355215007817 0.0027776967811517705
iter 5 i 65 fit(s) 0.0029575327347003996 0.002839962043001694
iter 5 i 66 fit(s) 0.00299170924551336 0.0028605886918588413
iter 5 i 67 fit(s) 0.0030929061228807115 0.002922216654463485
iter 5 i 68 fit(s) 0.003063361861061448 0.0029039666554073035
iter 5 i 69 fit(s) 0.003107748393959049 0.00293006604081368
iter 5 i 70 fit(s) 0.002944439560273835 0.002832230740920973
iter 5 i 71 fit(s) 0.003240702072092701 0.0030131164195636054
iter 5 i 72 fit(s) 0.002790548815769322 0.002741630628396695
iter 5 i 73 fit(s) 0.0031056467286425674 0.0029301815670637558
iter 5 i 74 fit(s) 0.002816062672836116 0.0027563756197671934
iter 5 i 75 f

...

particle members [31, 127, 145, 159, 169, 192, 193, 209, 236, 237, 244, 253, 320, 322, 372, 392, 403, 405, 414, 425, 464, 484, 493, 510, 595, 732, 870, 986, 1127, 1238, 1273, 1282, 1299, 1783, 1979, 2123, 2600, 2685, 2777, 2795, 2875, 2910, 2978, 2985, 3132, 3147, 3155, 3194, 3247, 3286, 3324, 3382, 3481, 3531, 3557, 3603, 3642, 3744, 3870, 3885, 3908, 3929, 3942, 3951, 4004, 4087, 4128, 4208, 4213, 4214, 4290, 4310, 4319, 4357, 4445, 4498, 4648, 4658, 4695, 4799, 4811, 4978, 5119, 5142, 5235, 5242, 5270, 5286, 5290, 5307, 5318, 5320, 5326, 5340, 5343, 5366, 5368, 5382, 5427, 5445, 5625, 5753, 5823, 5955, 5957, 5962, 6025, 6191, 6218, 6263, 6295, 6341, 6350, 6358, 6414, 6418, 6423, 6427, 6430, 6439, 6440, 6448, 6451, 6477, 6535, 6581, 6650, 6663, 7052, 7058, 7192, 7272, 7315, 7326, 7360, 7380, 7400, 7537, 7547, 7616, 7623, 7724, 7920, 8009, 8415, 8548, 9136, 9192, 9195, 9268, 9393, 9416, 9420, 9449, 9468, 9501, 9556]
particle members [21, 119, 120, 263, 511, 582, 603, 841, 886, 924, 11

particle members [5, 44, 195, 247, 273, 305, 364, 365, 368, 396, 413, 432, 446, 486, 523, 558, 563, 583, 627, 705, 765, 867, 876, 900, 923, 991, 1063, 1066, 1100, 1116, 1134, 1142, 1184, 1195, 1209, 1253, 1279, 1379, 1410, 1418, 1528, 1537, 1598, 1606, 1627, 1657, 1736, 1745, 1766, 1770, 1843, 1956, 2081, 2095, 2469, 2507, 2527, 2572, 2605, 2610, 2612, 2616, 2660, 2722, 2737, 2740, 2746, 2755, 2773, 2790, 2810, 2819, 2836, 2876, 2877, 2919, 2936, 3024, 3027, 3037, 3060, 3103, 3108, 3187, 3229, 3250, 3260, 3261, 3269, 3316, 3394, 3425, 3523, 3560, 3636, 3699, 3710, 3721, 3723, 3750, 3807, 3847, 3880, 3935, 3950, 3984, 3987, 4002, 4003, 4097, 4125, 4135, 4140, 4177, 4218, 4285, 4314, 4326, 4339, 4368, 4461, 4510, 4550, 4668, 4820, 4858, 4913, 4933, 5163, 5248, 5254, 5265, 5273, 5285, 5380, 5468, 5505, 5535, 5563, 5565, 5576, 5579, 5620, 5623, 5638, 5652, 5665, 5670, 5674, 5678, 5680, 5737, 5750, 5764, 5766, 5773, 5785, 5789, 5792, 5821, 5822, 5855, 5858, 5868, 6071, 6139, 6144, 6211, 625

particle members [10, 34, 77, 85, 144, 180, 443, 640, 810, 815, 868, 874, 1000, 1133, 1471, 1475, 1607, 1638, 1692, 1742, 1776, 1852, 1898, 1915, 1930, 1932, 1943, 1949, 1959, 2121, 2279, 2320, 2748, 3029, 3044, 3059, 3121, 3188, 3191, 3193, 3197, 3635, 3660, 4180, 4248, 4292, 4538, 4557, 4568, 4580, 4589, 4590, 4592, 4614, 4630, 4631, 4645, 4651, 4656, 4665, 4719, 4866, 5094, 5112, 5134, 5144, 5225, 5398, 5459, 5496, 5554, 5658, 5689, 5844, 5845, 5852, 5857, 5906, 6033, 6068, 6106, 6558, 6606, 6707, 6716, 6897, 7237, 7374, 7461, 7503, 7708, 7905, 7908, 7910, 8049, 8226, 8329, 8371, 8375, 8398, 8571, 8762, 8784, 8787, 8926, 8993, 9216, 9542, 9558, 9625, 9677, 9810]
particle members [695, 1451, 1454, 1472, 2424, 2545, 2596, 2718, 2731, 2972, 3017, 3751, 3886, 4096, 4121, 4433, 4661, 5973, 6484, 6999, 7394, 7764, 7765, 7776, 7793, 7881, 8096, 8120, 8308, 8492, 8569, 8666, 8694, 8715, 8729, 8776, 8793, 8978, 9116, 9120, 9331, 9522, 9611]
particle members [208, 318, 677, 1713, 1911, 2071, 

particle members [6951, 6990, 7053, 9808]
particle members [351, 470, 478, 488, 495, 600, 690, 707, 737, 750, 849, 850, 888, 947, 950, 960, 992, 1226, 1235, 1243, 1328, 1332, 1370, 1375, 1384, 1385, 1404, 1416, 1430, 1431, 1440, 1513, 1517, 1550, 1563, 1564, 1567, 1605, 1614, 1618, 1661, 1673, 1764, 1787, 1811, 1828, 1849, 1858, 1877, 1892, 1920, 1928, 1935, 1937, 1953, 1961, 1969, 2037, 2044, 2192, 2204, 2255, 2433, 2483, 2680, 2713, 2915, 3070, 3081, 3163, 3175, 3182, 3183, 3192, 3198, 3212, 3223, 3292, 3340, 3358, 3410, 3623, 3697, 3822, 3833, 3839, 3928, 3952, 3958, 4038, 4101, 4174, 4184, 4341, 4362, 4396, 4400, 4455, 4478, 4709, 4976, 5037, 5078, 5141, 5231, 5312, 5385, 5389, 5399, 5530, 5533, 5610, 5611, 5621, 5711, 5735, 5758, 5798, 5972, 5981, 6001, 6057, 6067, 6089, 6131, 6168, 6290, 6319, 6337, 6395, 6521, 6737, 6860, 6872, 6937, 7011, 7027, 7041, 7082, 7087, 7173, 7196, 7208, 7265, 7354, 7370, 7639, 7656, 7785, 7792, 7796, 7835, 7849, 7957, 8014, 8025, 8029, 8076, 8094, 818

particle members [36, 474, 619, 676, 678, 680, 887, 934, 1461, 1502, 1574, 1806, 1960, 2013, 2682, 2796, 3035, 3395, 3411, 3611, 3652, 3654, 3855, 4503, 4912, 4942, 4972, 5026, 5126, 5193, 5197, 5200, 5335, 5338, 5470, 5648, 5685, 5703, 5763, 6246, 6266, 6444, 6519, 6575, 6678, 6691, 6693, 7098, 7450, 8132, 8173, 8199, 8298, 8509, 8559, 8757, 8868, 9663, 9684, 9720, 9748, 9867, 9883, 9993]
particle members [54, 366, 614, 1522, 2394, 2631, 3468, 3586, 3643, 3673, 4188, 4197, 5311, 5860, 5936, 6458, 6742, 7049, 7548, 7580, 7600, 7667, 7767, 7773, 7922, 8045, 8382, 8386, 8832, 8983, 9324, 9584, 9706, 9736]
particle members [47, 463, 611, 636, 689, 697, 796, 877, 1089, 1097, 1136, 1230, 1249, 1270, 1283, 1294, 1300, 1412, 1423, 1462, 1547, 1651, 1668, 1830, 1835, 1889, 1972, 1995, 2029, 2039, 2105, 2125, 2151, 2160, 2161, 2169, 2177, 2217, 2243, 2561, 2579, 2712, 2717, 2834, 2837, 2880, 3071, 3093, 3097, 3167, 3291, 3458, 3806, 3871, 3875, 4074, 4149, 4151, 4274, 4311, 4370, 4405, 4424, 45